# 3. Instrument Beam, Instrument Noise, and Filtering
### Original from Jeff McMahon and Renée Hložek 

We will now add noise and filter the map - so that we have a map with all astrophysical and instrumental effects.

In [ ]:
import os
import sys
import numpy as np

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from constants import *
from cmb_modules import *

In [ ]:
data = '../data/'
out = '../out/'

### Just some matplotlib and seaborn parameter tuning

In [ ]:
# Set axtick dimensions
major_size = 5
minor_size = 3
major_width = minor_width = 1
for tick in ['xtick', 'ytick']:
    mpl.rcParams[f'{tick}.major.size'] = major_size
    mpl.rcParams[f'{tick}.major.width'] = major_width
    mpl.rcParams[f'{tick}.minor.size'] = minor_size
    mpl.rcParams[f'{tick}.minor.width'] = minor_width
mpl.rcParams['text.usetex'] = True

## Section 3.1: Recalculating the results from the previous stages

In [ ]:
## Create a pure CMB map
# Read in the input CMB spectra
data = np.genfromtxt(data + 'CAMB_real_scalCls.dat',)
ell = data[:,0]
DlTT = data[:,1]
ell_err = data[:,2]
DlTT_err = data[:,3]

ell2d, ClTT2d, FT_2d, CMB_I = make_CMB_I_map(ell, DlTT, N_x=N_x, N_y=N_y,
                                             X_width=X_width, Y_width=Y_width, pix_size=pix_size,
                                             random_seed=42)

# Make a point source map by generating Poisson distributed background and
# adding the exponentially distributed sources over that
PSmap_p = poisson_source_component(N_x, N_y, pix_size, number_of_sources, amplitude_of_sources)
PSmap_e = exponential_source_component(N_x, N_y, pix_size, number_of_sources_EX, amplitude_of_sources_EX)
PSmap = PSmap_p + PSmap_e

# Make an SZ map
SZmap, SZcat, beta = SZ_source_component(N_x, N_y,
                                         X_width, Y_width, pix_size,
                                         number_of_SZ_clusters, mean_amplitude_of_SZ_clusters, SZ_beta, SZ_theta_core)

# Add them all together to get the sky map at a single freuqency
total_map = CMB_I + PSmap + SZmap

In [ ]:
plot_CMB_map(total_map, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=False, save_filename='CMB_sim_with_foreground.png',
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

## Section 3.2: Sky Map Convolved with a Beam

Telescopes suffer from diffraction which leads to finite resolution effects.  To account for this we generate a Gaussian beam pattern and convolve the map with it.

In [ ]:
# FWHM
beam_size_fwhp = 1.25
# Convolve the signal part of the map
CMB_I_convolved = convolve_map_with_gaussian_beam(total_map,
                                                  N_x, N_y,
                                                  beam_size_fwhp=beam_size_fwhp)

In [ ]:
plot_CMB_map(CMB_I_convolved, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=False,
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

This plot shows the simulated map that includes the CMB, point source, and SZ signals and is convolved with an instrumental beam.  The agreement with the real sky maps is now reasonable, but does not yet include instrumental noise.

<font color='red'>ORIGINAL NOTES: </font> There are three typical beam sizes for CMB experiments: 1) large anguar scale B-mode observatories which have ~30 arcminute beams; 2) medium scale observatorires such as the Planck satellite that has a ~3 arcminute beam, and 3) high resolution observatories such as ACT and SPT that have ~1 arcminute beams.

### Compare different beam FWHM values

#### FWHM = 20 arcmin

In [ ]:
# FWHM
beam_size_fwhp = 20
# Convolve the signal part of the map
CMB_I_convolved = convolve_map_with_gaussian_beam(total_map,
                                                  N_x, N_y,
                                                  beam_size_fwhp=beam_size_fwhp)

In [ ]:
plot_CMB_map(CMB_I_convolved, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=True, save_filename='CMB_sim_fwhm_{0}.png'.format(beam_size_fwhp.strip('.')),
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

#### FWHM = 3 arcmin

In [ ]:
# FWHM
beam_size_fwhp = 3
# Convolve the signal part of the map
CMB_I_convolved = convolve_map_with_gaussian_beam(total_map,
                                                  N_x, N_y,
                                                  beam_size_fwhp=beam_size_fwhp)

In [ ]:
plot_CMB_map(CMB_I_convolved, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=True, save_filename='CMB_sim_fwhm_{0}.png'.format(beam_size_fwhp.strip('.')),
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

#### FWHM = 0.5 arcmin

In [ ]:
# FWHM
beam_size_fwhp = 0.5
# Convolve the signal part of the map
CMB_I_convolved = convolve_map_with_gaussian_beam(total_map,
                                                  N_x, N_y,
                                                  beam_size_fwhp=beam_size_fwhp)

In [ ]:
plot_CMB_map(CMB_I_convolved, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=True, save_filename='CMB_sim_fwhm_{0}.png'.format(beam_size_fwhp.strip('.')),
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

## Section 3.3: Noise Map

Ground based CMB instruments suffer from several types of noise.  This includes: (1) White noise drawn from a gaussian distribution; (2) atmospheric noise which grows larger on large angular scales, this can be understood in terms of Kolmogorov turbulence; and (3) $1/f$ noise in the detectors.  Both atmospheric and detector $1/f$ manifest as correlated noise in maps space.  However, like the CMB these noise components are very nearly uncorrelated in Fourier space.  We will build these noise terms in Fourier space and display them in map space. 

We choose a white noise level of 10 $\mu$K-arcmin to approximate the current deep maps coming from ACTPol and SPTPol.  The $1/f$ noise is similar to what would be seen in an experiment at the south pole.  When turned on, this noise leads to the effect of "striping" in the maps.  We default to turning this component off since if it is included we would need to filter the maps before estimating the power spectrum.  (This is easy, but is not yet implemented).  The atmospheric noise (Kolmogorov turbulence) has a 2D spectrum similar to CMB leading to similar patterns in the maps.

In [ ]:
white_noise_level = 10         # White noise [micro K/arcmin]
atmospheric_noise_level = 0.1  # Multiply by zero to turn this off
one_over_f_noise_level = 0.2   # Multiply by zero to turn this off

In [ ]:
nrows = 1
ncols = 3
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*9, nrows*9),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(wspace=0.35)

noises = [
    gen_white_noise(N_x, N_y, pix_size, white_noise_level),       # White noise
    gen_atmospheric_noise(N_x, N_y, X_width, Y_width, pix_size,
                          atmospheric_noise_level),               # Atmoshperic noise
    gen_one_over_f_noise(N_x, N_y,
                         pix_size, one_over_f_noise_level)        # 1/f noise
]
noises_titles = [
    'White noise map',
    'Atmoshperic noise map',
    '1/f noise map along X direction',
]

for i in range(ncols):
    ax = axes[i]
    im = ax.imshow(np.real(noises[i]), vmin=-100, vmax=100,
                   cmap='Greys_r')
    im.set_extent([-X_width/2,X_width/2, -Y_width/2,Y_width/2])
    
    ax.set_title(noises_titles[i],
                 color='white', fontsize=axistitlesize, fontweight='bold', pad=10)
    ax.set_xlabel('Angle $[^\circ]$', color='white', fontsize=axislabelsize, fontweight='bold')
    ax.set_ylabel('Angle $[^\circ]$', color='white', fontsize=axislabelsize, fontweight='bold')
    ax.tick_params(axis='both', which='major', colors='white', labelsize=axisticksize, labelrotation=42, pad=10)

    # Create an axis on the right side of `axes`. The width of `cax` will be 2%
    # of `axes` and the padding between `cax` and axes will be fixed at 0.1 inch
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2%', pad=0.1)
    cbar = plt.colorbar(mappable=im, cax=cax)#, shrink=0.735, pad=0.02)
    cbar.ax.tick_params(labelsize=axiscbarfontsize-3, colors='white')
    cbar.set_label('Temperature [$\mu$K]', color='white',
                   fontsize=axiscbarfontsize, rotation=90, labelpad=5)

if not os.path.exists(out):
    os.makedirs(out)
plt.savefig(out + 'CMB_sim_noise_compare.png',
            format='png', dpi=200,
            facecolor='black', edgecolor='black',
            bbox_inches='tight')
    
plt.show()

In [ ]:
## make an instrument noise map
noise = make_noise_map(N_x, N_y,
                       X_width, Y_width, pix_size,
                       white_noise_level,
                       atmospheric_noise_level, one_over_f_noise_level)

In [ ]:
plot_CMB_map(noise, X_width, Y_width, c_min=c_min/4, c_max=c_max/4,
             cmap = 'Greys_r',
             save=True, save_filename='CMB_sim_noise_full.png',
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

This plot shows a realization of instrumental + atmospheric noise.  The atmospheric noise looks similar to the CMB since it has a similar 2d spectrum.  

## Section 3.4: Complete Simulated CMB Map

We complete our simulated CMB map by adding the simulated sky map convolved with the beam to our simulated noise map.   

In [ ]:
# Add noise to the signal
total_map_plus_noise = CMB_I_convolved + noise

In [ ]:
plot_CMB_map(total_map_plus_noise, X_width, Y_width, c_min=c_min, c_max=c_max,
             save=True, save_filename='CMB_sim_final.png',
             no_axis=False, no_grid=True, no_title=False,
             no_cbar=False)

This plot shows a complete simulated CMB map including the key astrophysical and instrumental effects.